# The Goal
We need to have a list of UUIDs for those surveys that need to be excluded.

# The Problem
In the `object_repr` column of data in the database there are 3 problems in the way of meeting our goal
1. Some entries start or end with a single slash `/`
    - > /5dad3904-1125-4058-8088-545922f4695d
2. Some entries include the URL, but we only want the UUID contained in the end of the URL
    - > https://eskola.tl/#forms/activity/fef0362b-871e-4409-b0d3-fa4df2f1e101
3. Some of the entries have an incomplete UUID. They are too short and therefore invalid.
    - > fef0362b-87
4. A UUID may occur many times in that collumn.  Our list can only contain a UUID once. No duplicates
    - > fef0362b-871e-4409-b0d3-fa4df2f1e101
    - > fef0362b-871e-4409-b0d3-fa4df2f1e101

## The Data

In [5]:
import psycopg2 as pg
import pandas.io.sql as psql
 
# get connected to the database
connection = pg.connect("dbname=edu_db")
# read data from database 
dataframe = psql.read_sql_query("SELECT username, action_time, action_flag, change_message, object_repr from django_admin_log inner join auth_user on auth_user.id = django_admin_log.user_id WHERE content_type_id = 14 and action_flag=3 order by action_time limit 100", connection)
#dataframe = psql.read_sql_query("SELECT  object_repr from django_admin_log inner join auth_user on auth_user.id = django_admin_log.user_id WHERE content_type_id = 14 and action_flag=3 order by action_time limit 100", connection)
dataframe.head()

,username,action_time,action_flag,change_message,object_repr
0,tomasiamendonca,2018-03-27 07:17:34.422133+00:00,3,,/5dad3904-1125-4058-8088-545922f4695d
1,pauloquefi,2018-08-02 06:13:19.521625+00:00,3,,fef0362b-871e-4409-b0d3-fa4df2f1e101
2,pauloquefi,2018-08-02 06:13:19.525123+00:00,3,,https://eskola.tl/#forms/activity/fef0362b-871...
3,pauloquefi,2018-08-02 06:14:03.384585+00:00,3,,90d87acb-0f80-4dd4-a1a8-876e8e01fdef
4,pauloquefi,2018-08-02 06:15:36.281016+00:00,3,,https://eskola.tl/#forms/activity/cae06dee-8e3...


In [6]:
# Get a dirty list of UUIDs from the dataframe's object representation column
dirt_list_of_representations = dataframe['object_repr'].tolist()

In [7]:
## Test Data
dirty_test_list = ["https://eskola.tl/#forms/activity/fef0362b-871e-4409-b0d3-fa4df2f1e102",
                   "https://eskola.tl/#forms/observation/fef0362b-871e-4409-b0d3-fa4df2f1e102",
                   "bsrvation/fef0362b-871e-4409-b0d3-fa4df2f1e102",
                   "/5dad3904-1125-4058-8088-545922f4695d",
                   "fef0362b-87",
                   "fef0362b-871e-4409-b0d3-fa4df2f1e101",
                   "fef0362b-871e-4409-b0d3-fa4df2f1e101"]

clean_test_list = ["fef0362b-871e-4409-b0d3-fa4df2f1e102",
                   "5dad3904-1125-4058-8088-545922f4695d",
                   "fef0362b-871e-4409-b0d3-fa4df2f1e101"]


In [8]:
dirty_test_list[0].split('/')[-1]

'fef0362b-871e-4409-b0d3-fa4df2f1e102'

## Solution #1

In [9]:
def clean_list_of_object_representations(list_of_representations):
    unclean_list = [dirty.split('/')[-1] for dirty in list_of_representations]      # Problems 1 and 2
    clean_list = [clean for clean in unclean_list if len(clean) == 36]              # Problem 3
    clean_set = set(clean_list)                                                     # Problem 4
    return list(clean_set)
               
                   
try:
    assert clean_list_of_object_representations(dirty_test_list) == clean_test_list
except:
    print("That didn't work.")
    print(clean_list_of_object_representations(dirty_test_list))

In [87]:
clean_list = clean_list_of_object_representations(dirt_list_of_representations)
print(len(clean_list), 'items in the clean list')
clean_list

94 items in the clean list


['9064a018-75b6-4cfd-903c-047b668dda2c',
 '7ae96a3f-69fb-4132-9501-7bbcad89f50e',
 '04dba94b-6a5f-4273-809b-7604ff45cf70',
 'e99ea190-0c99-471a-9efa-ca1114ce9691',
 '7c8d8bec-8595-4d11-a4b4-b29c3ff56a93',
 '4c88c46b-57e6-4e3c-8a82-fd99c9c88f82',
 'c7872546-ebc1-4468-8951-01ce4965ca58',
 'a7f88cb0-acd7-422a-8f4d-7441f10236e4',
 'c7dec55b-5081-41a7-9a1c-4bd3fcb753a6',
 'a41d55d6-cb3e-4bbe-8f7a-3f32db446919',
 '5c91e5ae-5b9e-4f52-a79d-bc1d12d07f98',
 '270baff6-b2b8-4816-94c5-59a723b2ba1b',
 '686ff081-7048-43f8-a393-8660dc824dc4',
 '8d7ac21e-6912-406d-b6a3-e346918da803',
 '42c52ecb-ba3c-47e8-a336-efe869a04e2c',
 '698548a0-1f13-4b2f-a01b-f04a1249268a',
 '64c22fc0-cb5b-4fd4-a66e-bc5108b8569f',
 'ba438e2b-32d5-4de9-9c36-4d7a7e075e9b',
 '106f3f8c-6c31-4abd-acce-45d15f3732d3',
 '9d08022e-54fa-41a9-8434-e573021423cb',
 'ee97ff95-7e91-41d7-83b5-50efd19456d4',
 'b33955a1-29e9-4de6-b335-9ba57e25123e',
 '80bb2a2d-8f2c-4ca3-8a60-5f8fb20ee0a0',
 'fef0362b-871e-4409-b0d3-fa4df2f1e101',
 'e94f3235-0baa-

## All in one place

In [10]:
import psycopg2 as pg
import pandas.io.sql as psql
 
# get connected to the database
connection = pg.connect("dbname=edu_db")
# read data from database 
dataframe = psql.read_sql_query("SELECT  object_repr from django_admin_log inner join auth_user on auth_user.id = django_admin_log.user_id WHERE content_type_id = 14 and action_flag=3 order by action_time limit 100", connection)
dirt_list_of_representations = dataframe['object_repr'].tolist()

def clean_list_of_object_representations(list_of_representations):
    unclean_list = [dirty.split('/')[-1:][0] for dirty in list_of_representations]  # Problems 1 and 2
    clean_list = [clean for clean in unclean_list if len(clean) == 36]              # Problem 3
    clean_set = set(clean_list)                                                     # Problem 4
    return list(clean_set)

clean_list_of_object_representations(dirt_list_of_representations)

['99ed547c-62ec-42a5-9733-3a0765a58a4b',
 '2c83c4d9-7e65-4fbd-ac8d-89416dbf48e1',
 'cc489d4d-3c56-4e23-866e-31db6c32f2b2',
 '9847c64b-16a0-46b5-b229-40bdafb4a579',
 '7c2a10bd-fbb7-4f8d-9185-596369dff76c',
 'a7f88cb0-acd7-422a-8f4d-7441f10236e4',
 '44b54902-3a43-44c3-b74d-4e7d3f91c7a4',
 '2bbd9f51-bab5-4fe4-87d2-2a79f9fb5220',
 'd0904b90-6d15-45dd-b016-02ed6f7ae06d',
 '4e891909-a403-4bd1-96ef-90497cdc540a',
 'b6feaf13-bf89-42c4-a876-7f6934776129',
 '5c91e5ae-5b9e-4f52-a79d-bc1d12d07f98',
 '8d7ac21e-6912-406d-b6a3-e346918da803',
 '655e3610-45d4-42bd-868b-2f61b614a1c3',
 'c3ba94d2-58d8-4547-8ff2-0f0eed15efa1',
 '250a3dce-71f6-4ba9-a815-6fe77e641235',
 '5b7d4b7a-ad99-44a5-923b-05295ba37f41',
 '1fbaebc4-60dc-4184-ae17-26acfc9c6035',
 'e73eba48-eec9-4002-a4af-7bd00bd9f734',
 'd87676d1-69ae-44bc-84af-84589408e5df',
 '641979c1-0680-4ab3-ba8a-c577060d1272',
 '456e28e2-d5f9-47ee-97b4-af931d24b7c5',
 '6542ae81-f90f-4526-ae62-0b2cfa378687',
 '21e3ba3e-c002-45d3-9689-190a5e3435d3',
 '6a32071f-da21-